# Library

In [69]:
import os
import sys
import glob
import re
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import numpy as np
from ccdproc import ImageFileCollection
from astropy.time import Time
from datetime import datetime, timezone, timedelta
from astropy import units as u


#	Generate Mask Images

In [70]:
from astropy.io import fits
from astropy.wcs import WCS

def get_image_center_coords(fits_filename):
    # FITS 파일 열기
    with fits.open(fits_filename) as hdul:
        # WCS 정보 읽기
        wcs = WCS(hdul[0].header)
        
        # 이미지 중심의 x, y 좌표 계산
        width = hdul[0].header['NAXIS1']
        height = hdul[0].header['NAXIS2']
        x_center = width / 2
        y_center = height / 2
        
        # 중심의 x, y 좌표를 WCS 좌표 (RA, Dec)로 변환
        ra_center, dec_center = wcs.pixel_to_world(x_center, y_center).ra.deg, wcs.pixel_to_world(x_center, y_center).dec.deg
        
    return ra_center, dec_center

In [71]:
def create_mask_images(input_image, mask_suffix="mask.fits"):
	data = fits.getdata(input_image)
	mask = np.zeros_like(data, dtype=int)
	mask[data == 0] = 1
	mask[data != 0] = 0
	mask_filename = input_image.replace("fits", mask_suffix)
	fits.writeto(mask_filename, mask.astype(np.int8), overwrite=True)
	return mask_filename

def combine_or_mask(in_mask_image, ref_mask_image, mask_suffix="all_mask.fits"):
	inmask = fits.getdata(in_mask_image)
	refmask = fits.getdata(ref_mask_image)
	mask = np.logical_or.reduce([inmask, refmask])
	mask_filename = in_mask_image.replace("mask.fits", mask_suffix)
	fits.writeto(mask_filename, mask.astype(np.int8), overwrite=True)
	return mask_filename

In [72]:
def convert_number(num):
    if num.is_integer():
        return int(num)
    else:
        return num

In [73]:
def group_images(time_list, threshold):
    groups = []
    index_groups = []
    current_group = [time_list[0]]
    current_index_group = [0]  # 시작 인덱스

    for i in range(1, len(time_list)):
        if time_list[i] - time_list[i-1] <= threshold:
            current_group.append(time_list[i])
            current_index_group.append(i)
        else:
            groups.append(current_group)
            index_groups.append(current_index_group)
            current_group = [time_list[i]]
            current_index_group = [i]

    groups.append(current_group)  # 마지막 그룹을 추가
    index_groups.append(current_index_group)  # 마지막 인덱스 그룹을 추가
    return groups, index_groups

In [74]:
def calc_mean_dateloc(dateloclist):

	# 문자열을 datetime 객체로 변환
	datetime_objects = [datetime.fromisoformat(t) for t in dateloclist]

	# datetime 객체를 POSIX 시간으로 변환
	posix_times = [dt.timestamp() for dt in datetime_objects]

	# 평균 POSIX 시간 계산
	mean_posix_time = np.mean(posix_times)

	# 평균 POSIX 시간을 datetime 객체로 변환
	mean_datetime = datetime.fromtimestamp(mean_posix_time)

	# 필요한 경우, datetime 객체를 ISOT 형식의 문자열로 변환
	mean_isot_time = mean_datetime.isoformat()
	return mean_isot_time


In [75]:
def extract_date_and_time(date_obs_str, round_seconds=False):
    """
    Extract date and time from the 'DATE-OBS' FITS header keyword value.

    Parameters:
    date_obs_str (str): The DATE-OBS string, usually in the format 'YYYY-MM-DDTHH:MM:SS.sss'
    round_seconds (bool): Whether to round the seconds to the nearest whole number

    Returns:
    str, str: Extracted date and time strings in 'YYYYMMDD' and 'HHMMSS' formats
    """
    # Convert the DATE-OBS string to an Astropy Time object
    time_obj = Time(date_obs_str)

    # Extract the date and time components
    date_str = time_obj.strftime('%Y%m%d')
    if round_seconds:
        time_str = time_obj.strftime('%H%M%S')
    else:
        time_str = f"{time_obj.datetime.hour:02}{time_obj.datetime.minute:02}{int(time_obj.datetime.second):02}"

    return date_str, time_str

# Path

In [76]:
path_base = '/large_data/factory'
path_ref = f'{path_base}/ref_frame'
path_find = '/home/gp/gppy/phot/gregoryfind_7DT.py'
path_subtraction = "/home/gp/gppy/legacy/deprecated_src/util/gregorysubt_7DT.py"
path_data = '/large_data/factory/test/subtraction_S240422ed'
path_skygrid = "/large_data/factory/skygrid"
path_config = '/home/gp/gppy/config'

In [77]:
skygrid_table = Table.read(f"{path_skygrid}/skygrid_tiles_7DS.csv")

# Test data

In [78]:
stacked_images = ['/large_data/factory/test/subtraction_S240422ed/calib_7DT11_T12401_20240423_030502_r_360.com.fits']

# Image Stacking

In [79]:
keywords_to_add = [
    "IMAGETYP",
    # "EXPOSURE",
    # "EXPTIME",
    # "DATE-LOC",
    # "DATE-OBS",
    "XBINNING",
    "YBINNING",
    "GAIN",
    "EGAIN",
    "XPIXSZ",
    "YPIXSZ",
    "INSTRUME",
    "SET-TEMP",
    "CCD-TEMP",
    "TELESCOP",
    "FOCALLEN",
    "FOCRATIO",
    "RA",
    "DEC",
    # "CENTALT",
    # "CENTAZ",
    # "AIRMASS",
    "PIERSIDE",
    "SITEELEV",
    "SITELAT",
    "SITELONG",
    "FWHEEL",
    "FILTER",
    "OBJECT",
    "OBJCTRA",
    "OBJCTDEC",
    "OBJCTROT",
    "FOCNAME",
    "FOCPOS",
    "FOCUSPOS",
    "FOCUSSZ",
    "ROWORDER",
    # "COMMENT",
    "_QUINOX",
    "SWCREATE"
]

In [80]:
#	Time to group
threshold = 300./(60*60*24) # [MJD]
t_group = 0.5/24 # 30 min
tile_name_pattern = r"T\d{5}$"

In [81]:
ic_cal = ImageFileCollection(path_data, glob_include='calib*0.fits', keywords='*')

In [82]:
obs = '7DT01'

In [83]:
hdr = fits.getheader(f"{path_data}/{ic_cal.summary['file'][0]}")

In [ ]:
# grouplist = []
# stacked_images = []
# for obj in np.unique(ic_cal.summary['object']):
# 	for filte in np.unique(ic_cal.filter(object=obj).summary['filter']):

# 		print(f"[{obj},{filte}]==============================")
		
# 		checklist = []
# 		_imagearr = ic_cal.filter(object=obj, filter=filte).summary['file']
# 		#	Check Number of All Images


# 		if len(_imagearr) > 0:
# 			_mjdarr = Time(ic_cal.filter(object=obj, filter=filte).summary['date-obs'], format='isot').mjd

# 			groups, index_groups = group_images(
# 				time_list=_mjdarr,
# 				threshold=threshold
# 				)

# 			print("Groups:", groups)
# 			print("Index Groups:", index_groups)

# 			for gg, (group, indx_group) in enumerate(zip(groups, index_groups)):
# 				print(f"[{gg:0>2}] {indx_group}")

# 				if len(group) == 0:
# 					print(f"{_imagearr[indx_group][0]} Single image exists")
# 				elif len(group) > 1:
# 					grouped_images = _imagearr[indx_group]
# 					print(f"{len(grouped_images)} images to stack")
# 					for ii, inim in enumerate(grouped_images):
# 						if ii == 0:	
# 							print(f"- {ii:0>4}: {inim} <-- Base Image")
# 						else:
# 							print(f"- {ii:0>4}: {inim}")
					
# 					#	Base Image for the Alignment
# 					baseim = grouped_images[0]
# 					basehdr = fits.getheader(baseim)
# 					# print(f"BASE IMAGE: {baseim}")
# 					basecat = baseim.replace('fits', 'cat')
# 					path_imagelist = f"{os.path.dirname(baseim)}/{os.path.basename(baseim).replace('fits', 'image.list')}"

# 					#	Images to Combine for SWarp
# 					f = open(path_imagelist, 'w')
# 					for inim in grouped_images:
# 						f.write(f"{inim}\n")
# 					f.close()

# 					#	Get Header info
# 					dateloclist = []
# 					mjdlist = []
# 					exptimelist = []
# 					airmasslist = []
# 					altlist = []
# 					azlist = []
# 					for _inim in grouped_images:
# 						#	Open Image Header
# 						with fits.open(inim) as hdulist:
# 							# Get the primary header
# 							header = hdulist[0].header
# 							mjdlist.append(Time(header['DATE-OBS'], format='isot').mjd)
# 							exptimelist.append(header['EXPTIME'])
# 							airmasslist.append(header['AIRMASS'])
# 							dateloclist.append(header['DATE-LOC'])
# 							altlist.append(header['CENTALT'])
# 							azlist.append(header['CENTAZ'])
# 					exptime_combined = convert_number(np.sum(exptimelist))
# 					mjd_combined = np.mean(mjdlist)
# 					jd_combined = Time(mjd_combined, format='mjd').jd
# 					dateobs_combined = Time(mjd_combined, format='mjd').isot
# 					airmass_combined = np.mean(airmasslist)
# 					dateloc_combined = calc_mean_dateloc(dateloclist)
# 					alt_combined = np.mean(altlist)
# 					az_combined = np.mean(azlist)

# 					#	Center Coordinate
# 					#	Tile OBJECT (e.g. T01026)
# 					if bool(re.match(tile_name_pattern, obj)):
# 						print(f"{obj} is 7DT SkyGrid. Use Fixed RA, Dec!")
# 						indx_skygrid = skygrid_table['tile'] == obj
# 						ra, dec = skygrid_table['ra'][indx_skygrid][0], skygrid_table['dec'][indx_skygrid][0]
# 						c_tile = SkyCoord(ra, dec, unit=u.deg)

# 						objra = c_tile.ra.to_string(unit=u.hourangle, sep=':', pad=True)
# 						objdec = c_tile.dec.to_string(unit=u.degree, sep=':', pad=True, alwayssign=True)
# 						pass
# 					#	Non-Tile OBJECT
# 					else:
# 						print(f"{obj} is pointed (RA, Dec)")
# 						objra = header['OBJCTRA']
# 						objdec = header['OBJCTDEC']

# 						objra = objra.replace(' ', ':')
# 						objdec = objdec.replace(' ', ':')
# 					center = f"{objra},{objdec}"

# 					datestr, timestr = extract_date_and_time(dateobs_combined)
# 					comim = f"{path_data}/calib_{obs}_{obj}_{datestr}_{timestr}_{filte}_{exptime_combined}.com.fits"

# 					#	Image Combine
# 					# swarpcom = f"swarp -c {path_config}/7dt_{n_binning}x{n_binning}.swarp @{path_imagelist} -IMAGEOUT_NAME {comim} -RESAMPLE_DIR {path_data} -CENTER_TYPE MANUAL -CENTER {center} -GAIN_KEYWORD EGAIN"
# 					swarpcom = f"swarp -c {path_config}/7dt.swarp @{path_imagelist} -IMAGEOUT_NAME {comim} -RESAMPLE_DIR {path_data} -CENTER_TYPE MANUAL -CENTER {center} -GAIN_KEYWORD EGAIN"
# 					print(swarpcom)
# 					os.system(swarpcom)

# 					#	Get Genenral Header from Base Image
# 					with fits.open(baseim) as hdulist:
# 						header = hdulist[0].header
# 						chdr = {key: header.get(key, None) for key in keywords_to_add}

# 					#	Put General Header Infomation on the Combined Image
# 					with fits.open(comim) as hdulist:
# 						data = hdulist[0].data
# 						header = hdulist[0].header
# 						for key in list(chdr.keys()):
# 							header[key] = chdr[key]

# 					#	Effective EGAIN
# 					N_combine = len(grouped_images)
# 					gain_default = hdr['EGAIN']
# 					effgain = (2/3)*N_combine*gain_default

# 					#	Additional Header Information
# 					keywords_to_update = {
# 						'EGAIN'   : (effgain,          'Effective EGAIN, [e-/ADU] Electrons per A/D unit'),
# 						'FILTER'  : (filte,            'Active filter name'),
# 						'DATE-OBS': (dateobs_combined, 'Time of observation (UTC) for combined image'),
# 						'DATE-LOC': (dateloc_combined, 'Time of observation (local) for combined image'),
# 						'EXPTIME' : (exptime_combined, '[s] Total exposure duration for combined image'),
# 						'EXPOSURE': (exptime_combined, '[s] Total exposure duration for combined image'),
# 						'CENTALT' : (alt_combined,     '[deg] Average altitude of telescope for combined image'),
# 						'CENTAZ'  : (az_combined,      '[deg] Average azimuth of telescope for combined image'),
# 						'AIRMASS' : (airmass_combined, 'Average airmass at frame center for combined image (Gueymard 1993)'),
# 						'MJD'     : (mjd_combined,     'Modified Julian Date at start of observations for combined image'),
# 						'JD'      : (jd_combined,      'Julian Date at start of observations for combined image'),
# 					}

# 					#	Header Update
# 					with fits.open(comim, mode='update') as hdul:
# 						# 헤더 정보 가져오기
# 						header = hdul[0].header

# 						# 여러 헤더 항목 업데이트
# 						for key, (value, comment) in keywords_to_update.items():
# 							header[key] = (value, comment)

# 						# 변경 사항 저장
# 						hdul.flush()
# 					stacked_images.append(comim)

[T11623,r]==============================
Groups: [[60423.08171211806, 60423.08312241898, 60423.08453177083]]
Index Groups: [[0, 1, 2]]
[00] [0, 1, 2]
3 images to stack
- 0000: /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_015739_r_120.fits <-- Base Image
- 0001: /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_015941_r_120.fits
- 0002: /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_120.fits
T11623 is 7DT SkyGrid. Use Fixed RA, Dec!
swarp -c /home/gp/gppy/config/7dt.swarp @/large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_015739_r_120.image.list -IMAGEOUT_NAME /large_data/factory/test/subtraction_S240422ed/calib_7DT01_T11623_20240423_020143_r_360.com.fits -RESAMPLE_DIR /large_data/factory/test/subtraction_S240422ed -CENTER_TYPE MANUAL -CENTER 08:05:34.88372093,-22:55:28.30188679 -GAIN_KEYWORD EGAIN


----- SWarp 2.41.5 started on 2024-11-01 at 23:08:40 with 4 threads

------- Output File calib_7DT01_T11623_20240423_020143_r_360.com.fits:
    "no ident"  WEIGHTED  no ext. header  10200x6800  32 bits (floats)
    Center: 08:05:34.88 -22:55:28.3   85.8'x57.2'  Scale: 0.505 ''/pixel
    Gain: 0 e-/ADU   Flux scaling (astrom/photom): 1 X / 1 X

-------------- File calib_7DT05_T11623_20240423_015739_r_120.fits:
    "T11623"  unweighted  no ext. header  9576x6388  32 bits (floats)
    Center: 08:05:41.55 -22:56:56.7   80.8'x53.9'  Scale: 0.506 ''/pixel
    Gain: 0.256 e-/ADU   Flux scaling (astrom/photom): 0.9959 X / 0.008339 X
    Background: 1783.128   RMS: 89.3865

-------------- File calib_7DT05_T11623_20240423_015941_r_120.fits:
    "T11623"  unweighted  no ext. header  9576x6388  32 bits (floats)
    Center: 08:05:41.52 -22:56:56.9   80.8'x53.9'  Scale: 0.506 ''/pixel
    Gain: 0.256 e-/ADU   Flux scaling (astrom/photom): 0.996 X / 0.008313 X
    Background: 1796.267   RMS: 88.99412

# Main

In [92]:
stacked_images = sorted(glob.glob(f"{path_data}/c*m.fits"))
stacked_images

['/large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.fits']

In [93]:
reference_images = []
sci_mask_images = []
ref_mask_images = []
all_mask_images = []

for stack_image in stacked_images:
	part = os.path.basename(stack_image).split("_")
	obj = part[2]
	filte = part[5]
	path_ref_frame = f"{path_ref}/{filte}"
	# _reference_images = []
	# for ref_src in ['7DT', 'PS1']: ref_PS1_T14548_00000000_000000_r_0.fits
	_reference_images_ps1 = glob.glob(f"{path_ref_frame}/ref_PS1_{obj}_*_*_{filte}_0.fits")
	_reference_images_7dt = glob.glob(f"{path_ref_frame}/ref_7DT_{obj}_*_*_{filte}_*.fits")
	_reference_images = _reference_images_7dt + _reference_images_ps1

	if len(_reference_images) > 0:
		ref_image = _reference_images[0]
		#	Run
		sci_mask_image = create_mask_images(stack_image)
		ref_mask_image = create_mask_images(ref_image)
		all_mask_image = combine_or_mask(sci_mask_image, ref_mask_image, mask_suffix="all_mask.fits")
	else:
		ref_image = None
		sci_mask_image = None
		ref_mask_image = None
		all_mask_image = None
	#	Save
	reference_images.append(ref_image)
	sci_mask_images.append(sci_mask_image)
	ref_mask_images.append(ref_mask_image)
	all_mask_images.append(all_mask_image)


In [64]:
print(f"Reference Images     ({len(reference_images)}): {reference_images}")
print(f"Science Image Mask   ({len(sci_mask_images)}): {sci_mask_images}")
print(f"Reference Image Mask ({len(ref_mask_images)}): {ref_mask_images}")
print(f"All Image Mask       ({len(all_mask_images)}): {all_mask_images}")

Reference Images     (1): ['/large_data/factory/ref_frame/r/ref_PS1_T11874_00000000_000000_r_0.fits']
Science Image Mask   (1): ['/large_data/factory/test/subtraction_S240422ed/calib_7DT01_T11874_20240423_012137_r_240.com.mask.fits']
Reference Image Mask (1): ['/large_data/factory/ref_frame/r/ref_PS1_T11874_00000000_000000_r_0.mask.fits']
All Image Mask       (1): ['/large_data/factory/test/subtraction_S240422ed/calib_7DT01_T11874_20240423_012137_r_240.com.all_mask.fits']


In [94]:
for inim, refim in zip(stacked_images, reference_images):
	inra, indec = get_image_center_coords(inim)
	refra, refdec = get_image_center_coords(refim)

	print(f"Input: {inra},{indec}")
	print(f"Ref  : {refra},{refdec}")

Input: 121.39527268364337,-22.92445816298297
Ref  : 121.39527268364337,-22.92445816298297


#	Image Subtraction

In [101]:
for ss, (inim, refim, inmask_image, refmask_image, allmask_image) in enumerate(zip(stacked_images, reference_images, sci_mask_images, ref_mask_images, all_mask_images)):
	if refim != None:
		#	Subtraction Command
		subtraction_com = f"python {path_subtraction} {inim} {refim} {inmask_image} {refmask_image} {allmask_image}"
		print(subtraction_com)
		os.system(subtraction_com)

python /home/gp/gppy/legacy/deprecated_src/util/gregorysubt_7DT.py /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.fits /large_data/factory/ref_frame/r/ref_PS1_T11623_00000000_000000_r_0.fits /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.mask.fits /large_data/factory/ref_frame/r/ref_PS1_T11623_00000000_000000_r_0.mask.fits /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.all_mask.fits
21471 selected from 28398 (75.6%)
56001 selected from 86691 (64.6%)
13385 sources matched
Output Image   : /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.subt.fits
Convolved Image: /large_data/factory/test/subtraction_S240422ed/ref_PS1_T11623_20240423_020143_r_0.conv.fits
hotpants -c t -n i -iu 60000 -il -2.222 -tu 5100000 -tl -20000 -inim /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.

Doing : /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.fits -
        /large_data/factory/ref_frame/r/ref_PS1_T11623_00000000_000000_r_0.fits =
        /large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.subt.fits
   Good templ data : -20000.0 -> 5100000.0
   Good image data : -2.2 -> 60000.0
Mallocing massive amounts of memory...
Region 0 pixels            : 1:1774,1:1774
 Vector Indices (buffered) : 0:1773,0:1773
 Vector Indices (good data): 0:1699,0:1699
Build stamp  : t    0 i    0 (grid coord  0  0)
Build stamp  : t    0 i    0 (grid coord  1  0)
Build stamp  : t    0 i    0 (grid coord  2  0)
Build stamp  : t    0 i    0 (grid coord  3  0)
Build stamp  : t    0 i    0 (grid coord  4  0)
Build stamp  : t    0 i    0 (grid coord  5  0)
Build stamp  : t    0 i    0 (grid coord  6  0)
Build stamp  : t    0 i    0 (grid coord  7  0)
Build stamp  : t    0 i    0 (grid coord  8  0)
Build stamp  : t    0

DS9 region file '/large_data/factory/test/subtraction_S240422ed/calib_7DT05_T11623_20240423_020143_r_360.com.ssf.reg' has been created.
Apply Final Mask on both SUBT & CONV images


In [ ]:
#	Outputs
hdim = inim.replace("fits", "subt.fits")

_hcim = f"{os.path.basename(refim).replace('fits', 'conv.fits')}"
dateobs = os.path.basename(inim).split("_")[3]
timeobs = os.path.basename(inim).split("_")[4]
part_hcim = _hcim.split("_")
part_hcim[3] = dateobs
part_hcim[4] = timeobs
hcim = f"{path_data}/{'_'.join(part_hcim)}"

#	Photometry Command for Subtracted Image
phot_subt_com = f"python {path_phot_sub} {hdim} {inmask_image}"
print(phot_subt_com)
os.system(phot_subt_com)
#	Transient Search Command
search_com = f"python {path_find} {inim} {refim} {hcim} {hdim}"
print(search_com)
os.system(search_com)